In [1]:
# Evaluation Pipeline: Time-Series Fine-Tuning & Confusion Matrix
import os
import sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, f1_score, confusion_matrix

# Configuration
PROJECT_ROOT = Path("..")
RESULT_DIR = PROJECT_ROOT / "Results/Experiment_Result/Model_Preds/6class_other_existing_420"
CM_DIR     = PROJECT_ROOT / "Results/Experiment_Result/Confusion_Matrix/6class_other_existing_420"
METHODS    = ['full', 'some', 'one']
TIMES      = [10, 30] + list(range(60, 481, 60))
CLASS_NAMES = ['Shower','Tooth_brushing','Washing_hands','Wiping','Vacuum_Cleaner','Other']
CLASS_LABELS= ["Shower","Tooth Brushing","Washing Hands","Wiping","Vacuum Cleaner","Other"]

# Ensure output directories
RESULT_DIR.mkdir(parents=True, exist_ok=True)
CM_DIR.mkdir(parents=True, exist_ok=True)

# Aggregate metrics per method, participant, time
metrics = {m: {'BA': {}, 'F1': {}} for m in METHODS}
for pid_dir in RESULT_DIR.iterdir():
    if not pid_dir.is_dir(): continue
    pid = pid_dir.name
    for method in METHODS:
        ba_dict, f1_dict = metrics[method]['BA'], metrics[method]['F1']
        ba_dict.setdefault(pid, {})
        f1_dict.setdefault(pid, {})
        for t in TIMES:
            csvs = list((pid_dir / f"finetuned_{method}_{t}").glob('*.csv'))
            if not csvs:
                ba_dict[pid][t] = np.nan
                f1_dict[pid][t] = np.nan
                continue
            df = pd.read_csv(csvs[0])
            df = df[df['y_true']!='Other']
            if df.empty:
                ba_dict[pid][t] = np.nan
                f1_dict[pid][t] = np.nan
                continue
            ba_dict[pid][t] = balanced_accuracy_score(df['y_true'], df['y_pred'])
            f1_dict[pid][t] = f1_score(df['y_true'], df['y_pred'], average='weighted')

# Save per-method CSVs
for method in METHODS:
    for metric in ['BA','F1']:
        df_m = pd.DataFrame.from_dict(metrics[method][metric], orient='index', columns=TIMES)
        df_m.index.name = 'Participant'
        out_path = RESULT_DIR / f"{method}_{metric.lower()}_timeseries.csv"
        df_m.to_csv(out_path)
        print(f"Saved {out_path}")

# Combined confusion matrix for a single method and time
def plot_confusion_for(method: str, time_sec: int):
    all_true, all_pred = [], []
    for pid_dir in RESULT_DIR.iterdir():
        csvs = list((pid_dir / f"finetuned_{method}_{time_sec}").glob('*.csv'))
        if not csvs: continue
        df = pd.read_csv(csvs[0])
        all_true.extend(df['y_true'])
        all_pred.extend(df['y_pred'])
    cm = confusion_matrix(all_true, all_pred, labels=CLASS_NAMES)
    cm_pct = (cm.astype(float) / cm.sum(axis=1)[:,None]) * 100
    cm_pct = np.nan_to_num(cm_pct)
    fig, ax = plt.subplots(figsize=(6,5))
    im = ax.imshow(cm_pct, cmap='Blues', vmin=0, vmax=100)
    ax.set_xticks(range(len(CLASS_LABELS)))
    ax.set_yticks(range(len(CLASS_LABELS)))
    ax.set_xticklabels(CLASS_LABELS, rotation=45, ha='right', fontsize=10)
    ax.set_yticklabels(CLASS_LABELS, fontsize=10)
    ax.set_title(f"Confusion Matrix: {method} @ {time_sec}s")
    thresh = cm_pct.max()/2
    for i in range(cm_pct.shape[0]):
        for j in range(cm_pct.shape[1]):
            color = 'white' if cm_pct[i,j]>thresh else 'black'
            ax.text(j,i, f"{cm_pct[i,j]:.2f}", ha='center', va='center', color=color, fontsize=9)
    plt.tight_layout()
    out_png = CM_DIR / f"cm_{method}_{time_sec}.png"
    fig.savefig(out_png)
    plt.close(fig)
    print(f"Saved confusion matrix to {out_png}")

# Example: plot for 'full' at 420s
if __name__=='__main__':
    plot_confusion_for('full', 420)

C:\Users\user\anaconda3\envs\tf_gpu_ver3\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\user\anaconda3\envs\tf_gpu_ver3\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\user\anaconda3\envs\tf_gpu_ver3\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\user\anaconda3\envs\tf_gpu_ver3\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\user\anaconda3\envs\tf_gpu_ver3\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("

Saved ..\Results\Experiment_Result\Model_Preds\6class_other_existing_420\full_ba_timeseries.csv
Saved ..\Results\Experiment_Result\Model_Preds\6class_other_existing_420\full_f1_timeseries.csv
Saved ..\Results\Experiment_Result\Model_Preds\6class_other_existing_420\some_ba_timeseries.csv
Saved ..\Results\Experiment_Result\Model_Preds\6class_other_existing_420\some_f1_timeseries.csv
Saved ..\Results\Experiment_Result\Model_Preds\6class_other_existing_420\one_ba_timeseries.csv
Saved ..\Results\Experiment_Result\Model_Preds\6class_other_existing_420\one_f1_timeseries.csv
Saved confusion matrix to ..\Results\Experiment_Result\Confusion_Matrix\6class_other_existing_420\cm_full_420.png
